In [1]:
import pandas as pd
from tqdm import tqdm

import tensorflow as tf

/Users/uladzimir/app/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
%run "preprocessing.ipynb"

In [3]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

<h3>Model</h3>

In [4]:
x  = tf.placeholder(tf.float32, shape=[None, 128])
y_ = tf.placeholder(tf.float32, shape=[None, 1])

W_fc1 = weight_variable([128, 256])
b_fc1 = bias_variable([256])

y = tf.matmul(x,W_fc1) + b_fc1

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

train_step = tf.train.GradientDescentOptimizer(0.25).minimize(cross_entropy)

In [5]:
hangover['train'].reset()
hangover['test'].reset()

for _ in tqdm(range(30)):
    batch = hangover['train'].next_batch(10)
    train_step.run(feed_dict={x: batch['x'], y_: batch['y']})

  0%|          | 0/30 [00:00<?, ?it/s]


ValueError: Cannot execute operation using `run()`: No default session is registered. Use `with sess.as_default():` or pass an explicit session to `run(session=sess)`